In [ ]:
!python -m spacy download en_core_web_lg


In [ ]:
import spacy

nlp = spacy.load("en_core_web_lg")

with open("ex1.txt", "r") as f:
    text = f.read()

doc = nlp(text)


for ent in doc.ents:
    print(ent.text, ent.label_)

In [ ]:
# display
from spacy import displacy
displacy.render(doc, style ="ent")

In [8]:
# using json file

import json
with open("annotations.json", "r") as f:
    data = json.load(f)

In [18]:
data

{'classes': ['ADMIN_1', 'ADMIN_2', 'DISASTER'],
 'annotations': [['On 01 April 2017 at 3:00 pm, Gatsibo district located in the Eastern Province of Rwanda experienced heavy rainfall associated with heavy storms, which resulted in destruction of houses and community farm lands in Kiramuruzi Sector Nyabisindu Cell. The affected area is located 36 kilometers from Gatsibo District, 40 kilometers from the Eastern Province office and 70 kilometers from the City of Kigali.\r',
   {'entities': [[29, 36, 'ADMIN_2'],
     [61, 77, 'ADMIN_1'],
     [131, 143, 'DISASTER'],
     [213, 223, 'ADMIN_2'],
     [296, 303, 'ADMIN_2'],
     [337, 353, 'ADMIN_1'],
     [396, 402, 'ADMIN_1']]}],
  ['\r', {'entities': []}],
  ['In the beginning of October 2016, an upsurge of violence in the northern area of Myanmar’s Rakhine State led to mass displacement amongst the local population. The violence took place against a backdrop of decades of protracted tension and intercommunal violence between Rakhine and Mu

In [19]:
data['annotations']
# data.keys()

[['On 01 April 2017 at 3:00 pm, Gatsibo district located in the Eastern Province of Rwanda experienced heavy rainfall associated with heavy storms, which resulted in destruction of houses and community farm lands in Kiramuruzi Sector Nyabisindu Cell. The affected area is located 36 kilometers from Gatsibo District, 40 kilometers from the Eastern Province office and 70 kilometers from the City of Kigali.\r',
  {'entities': [[29, 36, 'ADMIN_2'],
    [61, 77, 'ADMIN_1'],
    [131, 143, 'DISASTER'],
    [213, 223, 'ADMIN_2'],
    [296, 303, 'ADMIN_2'],
    [337, 353, 'ADMIN_1'],
    [396, 402, 'ADMIN_1']]}],
 ['\r', {'entities': []}],
 ['In the beginning of October 2016, an upsurge of violence in the northern area of Myanmar’s Rakhine State led to mass displacement amongst the local population. The violence took place against a backdrop of decades of protracted tension and intercommunal violence between Rakhine and Muslim communities in Rakhine.The violence was sparked by a series of coord